In [1]:
from data import Corpus
import trainer
from utils import batchify
import json
import tensorflow as tf
import numpy as np

In [2]:
with open('wiki_bak/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('wiki_bak/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
VERSION = 102
params = dict(
    model_configs = {
      "rnn_layers":[
          {
             "units": 1024,
             "drop_i": 0.05,
             "wdrop": 0.1,
             "drop_o": 0.05
          },
          {
             "units": 1024,
             "wdrop": 0.1,
             "drop_o": 0.05
          },
          {
             "units": 1024,
             "drop_o": 0.05,
             "wdrop": 0.1
          }
],
       "vocab_size": len(word2idx) + 1,
       "drop_e": 0.0,
       "char_vocab_size": len(char2idx) + 1,
       "char_cnn_options": {
           "layers": [
                [1, 16],
                [2, 16],
                [3, 32],
                [4, 64],
                [5, 128],
                [6, 256],
                [7, 512]
           ],
           "n_highways": 2
       },
       "char_vec_size": 16,
       "projection_dims": 512,
       "skip_connection": True
    },
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 10240,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)


# In[12]:


my_trainer = trainer.Trainer(**params)

my_trainer.logger.info('Trainer params {}'.format(params))
# In[13]:


tf.reset_default_graph()
my_trainer.build()

Trainer params {'model_configs': {'rnn_layers': [{'units': 1024, 'drop_i': 0.05, 'wdrop': 0.1, 'drop_o': 0.05}, {'units': 1024, 'wdrop': 0.1, 'drop_o': 0.05}, {'units': 1024, 'drop_o': 0.05, 'wdrop': 0.1}], 'vocab_size': 153360, 'drop_e': 0.0, 'char_vocab_size': 1005, 'char_cnn_options': {'layers': [[1, 16], [2, 16], [3, 32], [4, 64], [5, 128], [6, 256], [7, 512]], 'n_highways': 2}, 'char_vec_size': 16, 'projection_dims': 512, 'skip_connection': True}, 'optimizer': <class 'tensorflow.python.training.gradient_descent.GradientDescentOptimizer'>, 'negative_samples': 10240, 'wdecay': 1.2e-06, 'alpha': 1e-06, 'beta': 1e-06, 'clip_norm': 0.3, 'bptt': 70, 'use_ema': True, 'save_freq': 1000, 'log_path': '102/logs', 'train_summary_dir': '102/train_summary/', 'test_summary_dir': '102/test_summary/', 'checkpoint_dir': '102/checkpoints/'}


INFO:tensorflow:Restoring parameters from 102/checkpoints/train/model.cpkt-557539


Restoring parameters from 102/checkpoints/train/model.cpkt-557539


In [3]:
tf.global_variables()

[<tf.Variable 'LanguageModel/decode_W:0' shape=(512, 153360) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/decode_b:0' shape=(153360,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/embedding_weight:0' shape=(1005, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/kernel:0' shape=(1, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/kernel:0' shape=(2, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/kernel:0' shape=(3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/kernel:0' shape=(4, 16, 64) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/bias:0' s

In [4]:
with open('wiki_bak/valid_char.npy', 'rb') as inp:
    test_char = np.load(inp)
with open('wiki_bak/valid_word.npy', 'rb') as inp:
    test_word = np.load(inp)
print(len(test_word), len(test_char))

11875464 11875464


In [5]:
test_word = batchify(test_word, 36).T
test_char = batchify(test_char, 36).T

In [6]:
my_trainer.evaluate_step(my_trainer.model_test, test_word, test_char)

Evaluate loss 3.019859790802002, time 5.425522327423096
Evaluate loss 2.81145977973938, time 5.859468460083008
Evaluate loss 2.82240891456604, time 6.3112568855285645
Evaluate loss 2.8563456535339355, time 6.767946243286133
Evaluate loss 2.8203601837158203, time 7.227608919143677
Evaluate loss 3.0015323162078857, time 7.6357667446136475
Evaluate loss 2.8956947326660156, time 8.042110443115234
Evaluate loss 2.8366265296936035, time 8.483861923217773
Evaluate loss 2.831665277481079, time 8.88953709602356
Evaluate loss 2.7678914070129395, time 9.297196388244629
Evaluate loss 3.084023952484131, time 9.706068515777588
Evaluate loss 3.0977540016174316, time 10.115173578262329
Evaluate loss 3.193277597427368, time 10.51998496055603
Evaluate loss 2.973973035812378, time 10.924056053161621
Evaluate loss 2.8764352798461914, time 11.331816911697388
Evaluate loss 2.9313571453094482, time 11.736921310424805
Evaluate loss 2.881516218185425, time 12.146650791168213
Evaluate loss 2.9267265796661377, t

Evaluate loss 2.9558794498443604, time 65.18383359909058
Evaluate loss 3.013742685317993, time 65.59422636032104
Evaluate loss 2.956793785095215, time 66.00480508804321
Evaluate loss 2.9195053577423096, time 66.4103741645813
Evaluate loss 2.883286714553833, time 66.8243100643158
Evaluate loss 2.888741970062256, time 67.23353886604309
Evaluate loss 2.807358980178833, time 67.64281940460205
Evaluate loss 2.8328728675842285, time 68.116783618927
Evaluate loss 2.81184983253479, time 68.52368259429932
Evaluate loss 2.8136188983917236, time 68.93080687522888
Evaluate loss 2.833979368209839, time 69.34200048446655
Evaluate loss 2.9900052547454834, time 69.75039196014404
Evaluate loss 3.0693933963775635, time 70.1595549583435
Evaluate loss 3.0084011554718018, time 70.56970691680908
Evaluate loss 3.067098379135132, time 70.97634410858154
Evaluate loss 3.1362435817718506, time 71.38515973091125
Evaluate loss 3.063744068145752, time 71.79368019104004
Evaluate loss 2.9460806846618652, time 72.2062

Evaluate loss 2.8845298290252686, time 124.54308843612671
Evaluate loss 2.9849326610565186, time 124.95384931564331
Evaluate loss 2.9706640243530273, time 125.36636209487915
Evaluate loss 2.903981924057007, time 125.77492499351501
Evaluate loss 2.853710889816284, time 126.18371224403381
Evaluate loss 2.839141607284546, time 126.59319114685059
Evaluate loss 2.7847492694854736, time 127.00235891342163
Evaluate loss 2.861274480819702, time 127.41944479942322
Evaluate loss 2.9929184913635254, time 127.82829141616821
Evaluate loss 2.998671531677246, time 128.23937940597534
Evaluate loss 3.129183292388916, time 128.64736938476562
Evaluate loss 3.0347023010253906, time 129.05746698379517
Evaluate loss 2.8975396156311035, time 129.46684098243713
Evaluate loss 2.9654855728149414, time 129.87578868865967
Evaluate loss 2.8894498348236084, time 130.28716349601746
Evaluate loss 2.888753652572632, time 130.69575548171997
Evaluate loss 2.9096364974975586, time 131.10429096221924
Evaluate loss 2.97341

Evaluate loss 2.9089622497558594, time 183.67581391334534
Evaluate loss 3.1136128902435303, time 184.0831491947174
Evaluate loss 3.099003791809082, time 184.49553036689758
Evaluate loss 3.02860689163208, time 184.90420770645142
Evaluate loss 3.16546368598938, time 185.313978433609
Evaluate loss 3.051903486251831, time 185.7243685722351
Evaluate loss 2.955197334289551, time 186.13600778579712
Evaluate loss 2.9862060546875, time 186.5480420589447
Evaluate loss 2.924147367477417, time 186.95702958106995
Evaluate loss 2.9792332649230957, time 187.36669063568115
Evaluate loss 3.033582925796509, time 187.77509999275208
Evaluate loss 3.119357109069824, time 188.1853747367859
Evaluate loss 3.0447428226470947, time 188.59943222999573
Evaluate loss 3.000734329223633, time 189.0121510028839
Evaluate loss 2.942568778991699, time 189.42434453964233
Evaluate loss 2.942013740539551, time 189.8383285999298
Evaluate loss 2.865034818649292, time 190.24584937095642
Evaluate loss 2.877256155014038, time 1

Evaluate loss 2.709611415863037, time 242.83288168907166
Evaluate loss 2.7011923789978027, time 243.24434089660645
Evaluate loss 2.8184425830841064, time 243.65505862236023
Evaluate loss 2.666860342025757, time 244.06609535217285
Evaluate loss 2.746994733810425, time 244.48079442977905
Evaluate loss 2.772196054458618, time 244.89232301712036
Evaluate loss 2.63167667388916, time 245.30260276794434
Evaluate loss 2.868447780609131, time 245.71329808235168
Evaluate loss 2.961261749267578, time 246.12584495544434
Evaluate loss 3.043091297149658, time 246.5340657234192
Evaluate loss 2.7888705730438232, time 246.94244241714478
Evaluate loss 2.9630162715911865, time 247.35479140281677
Evaluate loss 2.8879129886627197, time 247.76778554916382
Evaluate loss 2.8413171768188477, time 248.17547821998596
Evaluate loss 2.9436140060424805, time 248.58682680130005
Evaluate loss 2.9633827209472656, time 248.99422550201416
Evaluate loss 3.0439517498016357, time 249.4071753025055
Evaluate loss 2.943011760

...

Evaluate loss 2.958550453186035, time 1672.9396433830261
Evaluate loss 3.0328805446624756, time 1673.3509547710419
Evaluate loss 2.929936647415161, time 1673.7610688209534
Evaluate loss 3.0694801807403564, time 1674.1730437278748
Evaluate loss 2.9804439544677734, time 1674.5832142829895
Evaluate loss 3.0134024620056152, time 1674.9923281669617
Evaluate loss 3.051262855529785, time 1675.405793428421
Evaluate loss 2.920346260070801, time 1675.816563129425
Evaluate loss 3.0222978591918945, time 1676.228836297989
Evaluate loss 3.000462293624878, time 1676.642665863037
Evaluate loss 3.029587984085083, time 1677.0561311244965
Evaluate loss 2.8800878524780273, time 1677.4650793075562
Evaluate loss 2.908879280090332, time 1677.8790311813354
Evaluate loss 2.8350939750671387, time 1678.2958407402039
Evaluate loss 2.812608003616333, time 1678.711086511612
Evaluate loss 2.9226720333099365, time 1679.1213500499725
Evaluate loss 2.975231170654297, time 1679.5310425758362
Evaluate loss 2.940375089645

Evaluate loss 2.8112106323242188, time 1732.2375195026398
Evaluate loss 2.8319437503814697, time 1732.651061296463
Evaluate loss 2.8626906871795654, time 1733.0597887039185
Evaluate loss 2.969214916229248, time 1733.4703917503357
Evaluate loss 2.8932149410247803, time 1733.8807668685913
Evaluate loss 2.899705171585083, time 1734.2929859161377
Evaluate loss 2.961475372314453, time 1734.7074196338654
Evaluate loss 2.9757180213928223, time 1735.1207592487335
Evaluate loss 2.9027090072631836, time 1735.5312271118164
Evaluate loss 2.833118200302124, time 1735.9420418739319
Evaluate loss 3.0459647178649902, time 1736.3577971458435
Evaluate loss 2.8928868770599365, time 1736.7690253257751
Evaluate loss 2.890073776245117, time 1737.1798160076141
Evaluate loss 2.9191389083862305, time 1737.596988916397
Evaluate loss 2.7938907146453857, time 1738.0102682113647
Evaluate loss 2.916156053543091, time 1738.4234292507172
Evaluate loss 3.011042833328247, time 1738.8389098644257
Evaluate loss 2.9641559

Evaluate loss 2.9175424575805664, time 1791.6187603473663
Evaluate loss 2.961914539337158, time 1792.0303933620453
Evaluate loss 2.942373037338257, time 1792.4415555000305
Evaluate loss 3.0872879028320312, time 1792.8571696281433
Evaluate loss 2.9153220653533936, time 1793.2714896202087
Evaluate loss 2.8559834957122803, time 1793.6834816932678
Evaluate loss 2.7531161308288574, time 1794.0961611270905
Evaluate loss 2.821420907974243, time 1794.5108532905579
Evaluate loss 2.826486110687256, time 1794.9224092960358
Evaluate loss 2.7136428356170654, time 1795.3350546360016
Evaluate loss 2.805366039276123, time 1795.7446010112762
Evaluate loss 3.0181775093078613, time 1796.1530849933624
Evaluate loss 2.8337490558624268, time 1796.568484544754
Evaluate loss 2.8451433181762695, time 1796.9803609848022
Evaluate loss 2.8295509815216064, time 1797.3906450271606
Evaluate loss 2.918408155441284, time 1797.8009753227234
Evaluate loss 2.8562753200531006, time 1798.2149031162262
Evaluate loss 2.83363

Evaluate loss 2.742482900619507, time 1850.5251035690308
Evaluate loss 2.940753698348999, time 1850.9371223449707
Evaluate loss 2.9341397285461426, time 1851.3499505519867
Evaluate loss 2.8669559955596924, time 1851.7661864757538
Evaluate loss 2.746448278427124, time 1852.1745703220367
Evaluate loss 2.6954452991485596, time 1852.5844025611877
Evaluate loss 2.7454535961151123, time 1852.9946925640106
Evaluate loss 2.8159360885620117, time 1853.4028882980347
Evaluate loss 2.81606388092041, time 1853.8147249221802
Evaluate loss 2.749293804168701, time 1854.2220451831818
Evaluate loss 2.975771188735962, time 1854.6327319145203
Evaluate loss 3.0316572189331055, time 1855.0437498092651
Evaluate loss 2.997225046157837, time 1855.4577417373657
Evaluate loss 3.0647976398468018, time 1855.8668208122253
Evaluate loss 2.9165446758270264, time 1856.277527809143
Evaluate loss 2.89884090423584, time 1856.6905090808868
Evaluate loss 2.977220058441162, time 1857.109757900238
Evaluate loss 2.96194934844

Evaluate loss 2.986269235610962, time 1909.8715271949768
Evaluate loss 3.0304882526397705, time 1910.2857143878937
Evaluate loss 2.9118905067443848, time 1910.7015089988708
Evaluate loss 2.981360673904419, time 1911.1143221855164
Evaluate loss 2.997730016708374, time 1911.5326917171478
Evaluate loss 2.9174110889434814, time 1911.9437127113342
Evaluate loss 2.9240734577178955, time 1912.358566045761
Evaluate loss 2.994704484939575, time 1912.769278049469
Evaluate loss 2.9602086544036865, time 1913.177873134613
Evaluate loss 3.0028514862060547, time 1913.5890877246857
Evaluate loss 2.9335238933563232, time 1913.9992163181305
Evaluate loss 2.7589685916900635, time 1914.411681175232
Evaluate loss 2.8881404399871826, time 1914.820853471756
Evaluate loss 2.9181551933288574, time 1915.234682559967
Evaluate loss 2.739210844039917, time 1915.647558927536
Evaluate loss 2.815626621246338, time 1916.0595474243164
Evaluate loss 2.860352039337158, time 1916.4696698188782
Evaluate loss 2.952265501022